# A5 小書產生器（v8）— 好用的圖片上傳：封面／內頁／封底各有自己的選擇器

- ✅ **封面、內頁、封底** 各自有圖片上傳區（`FileUpload`）  
- ✅ 也提供 **URL/Base64 備援欄位**（若行動裝置上傳不穩，可貼連結或 base64，每行一張）  
- ✅ 下載走 **Base64 連結**，避開 403/424  
- ✅ 文字標記：`[14pt]`、`[圖片]`、`[圖片60%]`、`[左圖80%][右圖50%]`、`[無圖]`  
- ✅ 封面/封底若上傳了圖，會依你在該區文字中出現的 `[圖片]`/`[圖片60%]` 放入；沒有標記也會自動附在文後

**建議**：桌機用 Chrome/Firefox 體驗最佳；行動裝置若 `FileUpload` 不穩，請改用 URL/Base64 備援。


In [ ]:
# Imports
import io, re, base64, urllib.request, time
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

from docx import Document
from docx.shared import Mm, Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

from PIL import Image


In [ ]:
# Helpers

def ensure_rgb_jpeg(img_bytes, quality=90):
    im = Image.open(io.BytesIO(img_bytes))
    if im.mode not in ("RGB", "L"):
        im = im.convert("RGB")
    out = io.BytesIO()
    im.save(out, format="JPEG", quality=quality)
    return out.getvalue()

def parse_size_tag(text, default_pt=12):
    m = re.search(r"\[(\d+)pt\]\s*$", text)
    return int(m.group(1)) if m else default_pt

def strip_size_tag(text):
    return re.sub(r"\[\d+pt\]\s*$", "", text)

def has_skip_image_tag(text):
    return bool(re.search(r"\[無圖\]\s*$", text))

def strip_skip_image_tag(text):
    return re.sub(r"\[無圖\]\s*$", "", text)

def find_single_image_tag(text):
    m = re.search(r"\[(?:圖片)(\d+)?%?\]", text)
    if not m:
        return None
    pct = m.group(1)
    pct = int(pct) if pct else None
    return (pct, m.start(), m.end())

def find_double_image_tags(text):
    ml = re.search(r"\[左圖(\d+)?%?\]", text)
    mr = re.search(r"\[右圖(\d+)?%?\]", text)
    if not (ml and mr):
        return None
    lp = int(ml.group(1)) if ml.group(1) else None
    rp = int(mr.group(1)) if mr.group(1) else None
    return (lp, rp)

def set_section_to_a5(section):
    section.page_width = Mm(148)
    section.page_height = Mm(210)
    section.left_margin = Mm(15)
    section.right_margin = Mm(15)
    section.top_margin = Mm(15)
    section.bottom_margin = Mm(15)

def add_paragraph_with_size(doc, text, size_pt, align_center=False):
    p = doc.add_paragraph(text)
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER if align_center else WD_ALIGN_PARAGRAPH.LEFT
    for run in p.runs:
        run.font.size = Pt(size_pt)
    return p

def add_single_image_paragraph(doc, img_bytes, width_pct=None):
    page_w_mm = 148 - 15 - 15
    page_w_in = page_w_mm / 25.4
    if width_pct is None:
        width_in = page_w_in * 0.9
    else:
        width_in = page_w_in * (width_pct / 100.0)
        width_in = max(1.0, min(page_w_in, width_in))
    safe = ensure_rgb_jpeg(img_bytes)
    doc.add_picture(io.BytesIO(safe), width=Inches(width_in))

def add_double_image_table(doc, left_bytes, right_bytes, l_pct, r_pct):
    page_w_mm = 148 - 15 - 15
    page_w_in = page_w_mm / 25.4
    base_in = page_w_in / 2.0
    l_in = base_in if l_pct is None else base_in * (l_pct / 100.0)
    r_in = base_in if r_pct is None else base_in * (r_pct / 100.0)
    l_in = max(1.0, min(page_w_in, l_in))
    r_in = max(1.0, min(page_w_in, r_in))

    table = doc.add_table(rows=1, cols=2)
    table.autofit = True
    cell = table.rows[0].cells[0]
    run = cell.paragraphs[0].add_run()
    run.add_picture(io.BytesIO(ensure_rgb_jpeg(left_bytes)), width=Inches(l_in))
    cell = table.rows[0].cells[1]
    run = cell.paragraphs[0].add_run()
    run.add_picture(io.BytesIO(ensure_rgb_jpeg(right_bytes)), width=Inches(r_in))

def add_page_number_footer(section, start_from=1):
    footer = section.footer
    p = footer.paragraphs[0] if footer.paragraphs else footer.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    fld_simple = OxmlElement('w:fldSimple'); fld_simple.set(qn('w:instr'), 'PAGE')
    r = OxmlElement('w:r'); t = OxmlElement('w:t'); t.text = " "; r.append(t); fld_simple.append(r)
    p._p.append(fld_simple)
    sectPr = section._sectPr
    nodes = sectPr.xpath('./w:pgNumType')
    if nodes:
        pg = nodes[0]
    else:
        pg = OxmlElement('w:pgNumType'); sectPr.append(pg)
    pg.set(qn('w:start'), str(start_from))

def load_image_from_line(line: str) -> bytes:
    line = line.strip()
    if not line:
        return None
    if line.startswith('data:image'):
        head, b64 = line.split(',', 1)
        return base64.b64decode(b64)
    if line.startswith('http://') or line.startswith('https://'):
        with urllib.request.urlopen(line) as resp:
            return resp.read()
    # pure base64
    try:
        return base64.b64decode(line, validate=True)
    except Exception:
        raise ValueError("無法辨識的圖片輸入：請提供圖片 URL、data:base64 或純 base64。")

def make_data_uri_download(data: bytes, filename: str) -> HTML:
    b64 = base64.b64encode(data).decode()
    href = f'<a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.wordprocessingml.document;base64,{b64}">📥 點此下載 {filename}</a>'
    return HTML(href)


In [ ]:
# UI：封面 / 內頁 / 封底 各自的圖片來源（FileUpload + URL/Base64 備援）

# 文本
txt_cover = widgets.Textarea(
    value="我的 A5 小書\n（這是封面，會置中）\n[圖片60%]",
    description='封面文字',
    layout=widgets.Layout(width='100%', height='90px')
)
txt_body = widgets.Textarea(
    value=(
        "第一章……\n[圖片]\n\n"
        "第二章……\n[左圖80%][右圖60%]\n"
        "若不想消耗圖片，即使段落含[圖片]，可在尾端加[無圖]\n"
    ),
    description='內頁文字',
    layout=widgets.Layout(width='100%', height='180px')
)
txt_back = widgets.Textarea(
    value="封底簡介……\n[圖片60%]",
    description='封底文字',
    layout=widgets.Layout(width='100%', height='90px')
)

# 頁碼
chk_pagenum = widgets.Checkbox(value=True, description='內頁加頁碼')

# 圖片：封面（單張）
up_cover = widgets.FileUpload(accept='image/*', multiple=False, description='上傳封面圖（單張）')
cover_urls = widgets.Textarea(placeholder='封面圖 URL 或 base64，一行一張（通常只需一張）', layout=widgets.Layout(height='60px'))

# 圖片：內頁（多張，依上傳順序）
up_body = widgets.FileUpload(accept='image/*', multiple=True, description='上傳內頁圖（可多張）')
body_urls = widgets.Textarea(placeholder='內頁圖 URL 或 base64，每行一張（依先後使用）', layout=widgets.Layout(height='90px'))

# 圖片：封底（單張）
up_back = widgets.FileUpload(accept='image/*', multiple=False, description='上傳封底圖（單張）')
back_urls = widgets.Textarea(placeholder='封底圖 URL 或 base64，一行一張（通常只需一張）', layout=widgets.Layout(height='60px'))

btn = widgets.Button(description="生成 A5 小書（docx）", button_style='success')
out = widgets.Output()

display(widgets.VBox([
    widgets.HBox([widgets.VBox([txt_cover, txt_back]), widgets.VBox([chk_pagenum])]),
    widgets.Label('—— 內頁 ——'),
    txt_body,
    widgets.Label('—— 封面圖片 ——'),
    widgets.HBox([up_cover, cover_urls]),
    widgets.Label('—— 內頁圖片（依上傳/輸入先後） ——'),
    widgets.HBox([up_body, body_urls]),
    widgets.Label('—— 封底圖片 ——'),
    widgets.HBox([up_back, back_urls]),
    btn,
    out
]))


In [ ]:
# 組裝 DOCX（支援封面/內頁/封底三組圖片來源）

def gather_bytes_from_uploader(upld, allow_many=False):
    data = []
    for meta in upld.value:
        raw = meta.get('content', b'')
        if raw:
            data.append(raw)
        if (not allow_many) and data:
            break
    return data

def gather_bytes_from_textarea(txt, allow_many=False):
    lines = [ln.strip() for ln in txt.value.splitlines() if ln.strip()]
    data = []
    for ln in lines:
        try:
            b = load_image_from_line(ln)
            if b:
                data.append(b)
        except Exception:
            pass
        if (not allow_many) and data:
            break
    return data

def build_docx_with_cover_body_back():
    # 文字
    cover_text = txt_cover.value
    body_text  = txt_body.value
    back_text  = txt_back.value

    # 封面圖：先取上傳；若無再取 URL/base64
    cover_imgs = gather_bytes_from_uploader(up_cover, allow_many=False)
    if not cover_imgs:
        cover_imgs = gather_bytes_from_textarea(cover_urls, allow_many=False)

    # 內頁圖：上傳 + URL 列表合併
    body_imgs = gather_bytes_from_uploader(up_body, allow_many=True) + gather_bytes_from_textarea(body_urls, allow_many=True)

    # 封底圖
    back_imgs = gather_bytes_from_uploader(up_back, allow_many=False)
    if not back_imgs:
        back_imgs = gather_bytes_from_textarea(back_urls, allow_many=False)

    # 先建 DOCX 的封面
    doc = Document()
    set_section_to_a5(doc.sections[0])

    # 封面：文字 + 可選圖片
    size_pt = parse_size_tag(cover_text, default_pt=18)
    text = strip_size_tag(cover_text)
    single = find_single_image_tag(text)
    if single and cover_imgs:
        pct, s, e = single
        text_clean = (text[:s] + text[e:]).strip()
        if text_clean:
            add_paragraph_with_size(doc, text_clean, size_pt, align_center=True)
        add_single_image_paragraph(doc, cover_imgs[0], width_pct=pct)
    else:
        add_paragraph_with_size(doc, text.strip(), size_pt, align_center=True)
        if cover_imgs:
            add_single_image_paragraph(doc, cover_imgs[0], width_pct=60)

    # 內頁：新節
    doc.add_section()
    set_section_to_a5(doc.sections[-1])
    if chk_pagenum.value:
        add_page_number_footer(doc.sections[-1], start_from=1)

    # 內頁內容（與之前一致）
    paragraphs = [p.strip() for p in body_text.split('\n')]
    img_idx = 0
    for raw in paragraphs:
        if not raw:
            doc.add_paragraph("")
            continue
        size_pt = parse_size_tag(raw, default_pt=12)
        text = strip_size_tag(raw)
        skip_img = has_skip_image_tag(text)
        text = strip_skip_image_tag(text)

        dbl = find_double_image_tags(text)
        if dbl and (not skip_img) and (img_idx + 1 < len(body_imgs)):
            text_clean = re.sub(r"\[左圖\d*%?\]\s*\[右圖\d*%?\]", "", text).strip()
            if text_clean:
                add_paragraph_with_size(doc, text_clean, size_pt)
            l_pct, r_pct = dbl
            add_double_image_table(doc, body_imgs[img_idx], body_imgs[img_idx+1], l_pct, r_pct)
            img_idx += 2
            continue

        single = find_single_image_tag(text)
        if single and (not skip_img) and (img_idx < len(body_imgs)):
            pct, s, e = single
            text_clean = (text[:s] + text[e:]).strip()
            if text_clean:
                add_paragraph_with_size(doc, text_clean, size_pt)
            add_single_image_paragraph(doc, body_imgs[img_idx], width_pct=pct)
            img_idx += 1
            continue

        add_paragraph_with_size(doc, text, size_pt)

    # 封底：新節
    doc.add_section()
    set_section_to_a5(doc.sections[-1])
    size_pt = parse_size_tag(back_text, default_pt=12)
    text = strip_size_tag(back_text)
    single = find_single_image_tag(text)
    if single and back_imgs:
        pct, s, e = single
        text_clean = (text[:s] + text[e:]).strip()
        if text_clean:
            add_paragraph_with_size(doc, text_clean, size_pt, align_center=True)
        add_single_image_paragraph(doc, back_imgs[0], width_pct=pct)
    else:
        add_paragraph_with_size(doc, text.strip(), size_pt, align_center=True)
        if back_imgs:
            add_single_image_paragraph(doc, back_imgs[0], width_pct=60)

    # 封裝
    buf = io.BytesIO(); doc.save(buf); data = buf.getvalue()
    _ = Document(io.BytesIO(data))
    return data

def on_click(_):
    with out:
        clear_output()
        try:
            data = build_docx_with_cover_body_back()
            fname = f"a5_book_{int(time.time())}.docx"
            display(HTML('<div style="color:green;">完成！</div>'))
            display(make_data_uri_download(data, fname))
        except Exception as e:
            display(HTML(f'<div style="color:red;">失敗：{e}</div>'))

btn.on_click(on_click)
